In [2]:
# API KEY Loading
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from langchain_teddynote import logging

logging.langsmith("CH06-Memory")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH06-Memory


# Multi Turn 구성

In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.output_parsers import StrOutputParser

In [5]:
# 모델 정의/초기화 
model = ChatOpenAI(model_name='gpt-4o-mini')

# 대화형 프롬프트 정의 
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 Question-Answering 챗봇입니다. 주어진 질문에 대한 답변을 제공해주세요.",
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "#Question:\n{question}"),  # 사용자 입력을 변수로 사용
    ]
)

# 기본형 체인 구성 
chain = prompt | model | StrOutputParser()

In [6]:
# 대화를 저장하는 체인 구성 
# store : 세션 기록을 저장할 딕셔너리, 세션 종료 시 사라짐 (휘발성)
store = {}

# 세션ID 기반으로 세션 기록 가져오는 함수 
def get_session_history(session_ids):
    print(f"[대화 세션ID]: {session_ids}")

    if session_ids not in store:  
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]

# 대화를 기억하는 체인 구성 
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,  # 세션 기록을 가져오는 함수
    input_messages_key="question",  # 사용자의 질문이 템플릿 변수에 들어갈 key
    history_messages_key="chat_history",  # 기록 메시지의 키
)

In [7]:
# 첫번째 질문 
chain_with_history.invoke(
    {'question': '안녕하세요. 제 이름은 김철수 입니다.'},
    config={'configurable':{"session_id": "skala-001"}},
)

[대화 세션ID]: skala-001


'안녕하세요, 김철수님! 어떻게 도와드릴까요?'

In [8]:
# 두번째 질문 
chain_with_history.invoke(
    {'question': '제 이름 기억하세요?'},
    config={'configurable':{"session_id": "skala-001"}},
)

[대화 세션ID]: skala-001


'네, 김철수님이라고 말씀하셨습니다! 다른 질문이나 도움이 필요하시면 언제든지 말씀해 주세요.'

In [9]:
# 세션ID 변경하여 질문하면...
chain_with_history.invoke(
    {'question': '제 이름 기억하세요?'},
    config={'configurable':{"session_id": "skala-100"}},
)

[대화 세션ID]: skala-100


'죄송하지만, 이전 대화 내용을 기억할 수 없어서 이름을 알지 못합니다. 당신의 이름을 말씀해 주시면 기억하겠습니다!'

-----
** End of Documents **